<a href="https://colab.research.google.com/github/mkufel/CryptoTrader/blob/master/CryptoTrader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Dataset https://www.kaggle.com/sudalairajkumar/cryptocurrencypricehistory#ethereum_price.csv

In [2]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    data_path = '/content/drive/My Drive/ID2223/crypto_data/'
except:
  print("Failed to connect to Drive")

Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd

In [0]:
currencies = ["bitcoin", "bitcoin_cash", "bitconnect", "dash", "ethereum", "ethereum_classic", "iota", "litecoin", "monero", "nem", "neo", "numeraire", "omisego", "qtum", "ripple", "stratis" , "waves"]

In [5]:
df_test = pd.read_csv(f"{data_path}bitcoin_price.csv", names=['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Market Cap'])
df_test.head()

,Date,Open,High,Low,Close,Volume,Market Cap
0,Date,Open,High,Low,Close,Volume,Market Cap
1,"Feb 20, 2018",11231.80,11958.50,11231.80,11403.70,"9,926,540,000","189,536,000,000"
2,"Feb 19, 2018",10552.60,11273.80,10513.20,11225.30,"7,652,090,000","178,055,000,000"
3,"Feb 18, 2018",11123.40,11349.80,10326.00,10551.80,"8,744,010,000","187,663,000,000"
4,"Feb 17, 2018",10207.50,11139.50,10149.40,11112.70,"8,660,880,000","172,191,000,000"


In [49]:
# Get the date column for further joins, reverse the order
all_data = pd.DataFrame(data=df_test['Date'])[::-1]

for currency in currencies:
  # Read a file for the given currency
  df = pd.read_csv(f'{data_path}{currency}_price.csv', names=['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Market Cap'])
  df = df.iloc[1:]

  # Rename columns adding currency names in headers
  df.rename(columns={
      'Open': f'Open_{currency}',
      'High': f'High_{currency}',
      'Low': f'Low_{currency}',
      'Close': f'Close_{currency}',
      'Volume': f'Volume_{currency}',
      'Market Cap': f'Market_cap_{currency}'},
      inplace=True)
  
  # Merge the renamed data frame with all_data on the Date column
  all_data = pd.merge(all_data, df, on='Date')

print(all_data.head(2))
for column in all_data.columns:
  print(column)

           Date Open_bitcoin  ... Volume_waves Market_cap_waves
0  Jul 23, 2017      2808.10  ...    2,608,730      390,012,000
1  Jul 24, 2017      2732.70  ...    1,813,130      364,582,000

[2 rows x 103 columns]
Date
Open_bitcoin
High_bitcoin
Low_bitcoin
Close_bitcoin
Volume_bitcoin
Market_cap_bitcoin
Open_bitcoin_cash
High_bitcoin_cash
Low_bitcoin_cash
Close_bitcoin_cash
Volume_bitcoin_cash
Market_cap_bitcoin_cash
Open_bitconnect
High_bitconnect
Low_bitconnect
Close_bitconnect
Volume_bitconnect
Market_cap_bitconnect
Open_dash
High_dash
Low_dash
Close_dash
Volume_dash
Market_cap_dash
Open_ethereum
High_ethereum
Low_ethereum
Close_ethereum
Volume_ethereum
Market_cap_ethereum
Open_ethereum_classic
High_ethereum_classic
Low_ethereum_classic
Close_ethereum_classic
Volume_ethereum_classic
Market_cap_ethereum_classic
Open_iota
High_iota
Low_iota
Close_iota
Volume_iota
Market_cap_iota
Open_litecoin
High_litecoin
Low_litecoin
Close_litecoin
Volume_litecoin
Market_cap_litecoin
Open_monero
H

In [0]:
TARGET_CURRENCY = 'waves'
SEQ_LEN_PAST = 30
SEQ_LEN_FUT = 3

In [54]:
all_data['Future'] = all_data[f"Close_{TARGET_CURRENCY}"].shift(-SEQ_LEN_FUT)
all_data.head()

,Date,Open_bitcoin,High_bitcoin,Low_bitcoin,Close_bitcoin,Volume_bitcoin,Market_cap_bitcoin,Open_bitcoin_cash,High_bitcoin_cash,Low_bitcoin_cash,Close_bitcoin_cash,Volume_bitcoin_cash,Market_cap_bitcoin_cash,Open_bitconnect,High_bitconnect,Low_bitconnect,Close_bitconnect,Volume_bitconnect,Market_cap_bitconnect,Open_dash,High_dash,Low_dash,Close_dash,Volume_dash,Market_cap_dash,Open_ethereum,High_ethereum,Low_ethereum,Close_ethereum,Volume_ethereum,Market_cap_ethereum,Open_ethereum_classic,High_ethereum_classic,Low_ethereum_classic,Close_ethereum_classic,Volume_ethereum_classic,Market_cap_ethereum_classic,Open_iota,High_iota,Low_iota,...,Close_neo,Volume_neo,Market_cap_neo,Open_numeraire,High_numeraire,Low_numeraire,Close_numeraire,Volume_numeraire,Market_cap_numeraire,Open_omisego,High_omisego,Low_omisego,Close_omisego,Volume_omisego,Market_cap_omisego,Open_qtum,High_qtum,Low_qtum,Close_qtum,Volume_qtum,Market_cap_qtum,Open_ripple,High_ripple,Low_ripple,Close_ripple,Volume_ripple,Market_cap_ripple,Open_stratis,High_stratis,Low_stratis,Close_stratis,Volume_stratis,Market_cap_stratis,Open_waves,High_waves,Low_waves,Close_waves,Volume_waves,Market_cap_waves,Future
0,"Jul 23, 2017",2808.10,2832.18,2653.94,2730.40,"1,072,840,000","46,231,100,000",555.89,578.97,411.78,413.06,"85,013",-,67.36,67.85,58.75,60.14,"2,132,900","410,971,000",195.02,195.74,187.26,191.03,"31,110,400","1,451,390,000",229.12,231.68,217.72,225.95,"649,615,000","21,418,300,000",16.52,16.62,15.37,15.78,"76,647,400","1,549,590,000",0.273693,0.280241,0.258485,...,8.57,"29,915,100","431,881,000",36.59,37.54,32.41,32.89,"515,553","44,748,100",0.878973,1.59,0.850754,1.53,"15,198,100","86,413,600",7.55,7.62,6.14,6.64,"23,752,700","385,277,000",0.193568,0.204121,0.188722,0.195058,"195,682,000","7,411,980,000",6.80,6.96,5.94,6.17,"12,522,100","669,425,000",3.90,3.97,3.50,3.65,"2,608,730","390,012,000",2.74
1,"Jul 24, 2017",2732.70,2777.26,2699.19,2754.86,"866,474,000","44,995,600,000",412.58,578.89,409.21,440.70,"190,952",-,61.33,68.22,58.44,67.02,"3,991,940","374,541,000",191.11,216.62,187.29,204.02,"90,278,800","1,422,690,000",226.28,227.54,222.45,224.71,"431,691,000","21,158,500,000",15.84,16.02,15.56,15.88,"38,370,000","1,486,560,000",0.265136,0.273468,0.242731,...,8.46,"21,283,700","430,471,000",33.01,36.16,32.04,33.69,"551,830","40,370,800",1.51,1.61,1.31,1.52,"12,526,000","148,581,000",6.67,7.30,6.46,7.26,"12,601,300","339,927,000",0.194979,0.195279,0.189835,0.192002,"56,632,800","7,466,020,000",6.16,6.35,5.65,5.77,"10,593,700","607,093,000",3.65,3.71,3.39,3.52,"1,813,130","364,582,000",2.75
2,"Jul 25, 2017",2757.50,2768.08,2480.96,2576.48,"1,460,090,000","45,410,100,000",441.35,541.66,338.09,406.90,"524,908",-,67.01,67.01,60.14,63.65,"3,566,120","413,094,000",204.04,212.40,189.03,195.52,"84,970,400","1,519,340,000",224.37,225.32,195.84,206.71,"1,095,760,000","20,985,200,000",15.89,15.93,14.49,14.92,"74,645,500","1,491,550,000",0.266827,0.269072,0.237998,...,7.35,"19,480,400","423,876,000",33.69,33.89,25.92,27.29,"460,800","41,210,000",1.53,1.55,1.12,1.23,"8,132,390","150,667,000",7.31,7.61,5.68,6.13,"22,782,500","372,639,000",0.192280,0.192734,0.169797,0.178104,"166,807,000","7,370,540,000",5.76,5.92,4.31,4.82,"20,657,700","567,608,000",3.52,3.56,2.76,2.91,"2,462,600","351,753,000",3.00
3,"Jul 26, 2017",2577.77,2610.76,2450.80,2529.45,"937,404,000","42,455,000,000",407.08,486.16,321.79,365.82,"1,784,640",-,63.57,64.46,60.63,62.68,"3,436,370","398,636,000",195.49,200.88,189.33,194.37,"38,380,000","1,456,070,000",207.09,210.77,194.78,203.95,"717,692,000","19,373,400,000",14.86,15.08,14.22,14.77,"46,058,800","1,395,600,000",0.250571,0.283478,0.241237,...,7.17,"16,765,600","366,562,000",27.65,28.93,24.75,26.97,"323,792","33,814,100",1.23,1.29,1.10,1.21,"4,740,910","121,110,000",6.14,6.49,5.28,6.26,"20,824,500","362,203,000",0.177582,0.180872,0.169060,0.173822,"76,584,600","6,807,110,000",4.83,5.37,4.57,5.06,"12,565,800","475,864,000",2.92,2.99,2.50,2.74,"3,013,790","

In [90]:
# Add a column of target labels, if the rate rose in SEQ_LEN_FUT: 1, if fell: 0
all_data['Target'] = list(map(lambda past, future: int(float(future) > float(past)), all_data[f'Close_{TARGET_CURRENCY}'], all_data['Future']))
all_data[[f'Close_{TARGET_CURRENCY}', 'Future', 'Target']].head(5)

,Close_waves,Future,Target
0,3.65,2.74,0
1,3.52,2.75,0
2,2.91,3.00,1
3,2.74,2.87,1
4,2.75,2.72,0
